# Songs details statistics

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import datetime

## DataFrames

In [2]:
# Songs list (spotify_id, name, artists, album_name, album_release_date)
songlist = pd.read_csv('/home/hunor/hobby_projects/spotify/data/song_list.csv', sep=',')

# Song details (spotify_id, is_explicit, duration_ms, danceability, energy, ...)
songdetails = pd.read_csv('/home/hunor/hobby_projects/spotify/data/song_details.csv', sep=',')

# Song daily rank (spotify_id, daily_rank, daily_movement, weekly_movement, country, ...)
songdailyrank = pd.read_csv('/home/hunor/hobby_projects/spotify/data/song_daily_rank.csv', sep=',')

In [3]:
songlist.head()

,spotify_id,name,artists,album_name,album_release_date
0,2OzhQlSqBEmt7hmkYxfT6m,Fortnight (feat. Post Malone),Taylor Swift,THE TORTURED POETS DEPARTMENT,2024-04-18
1,2OzhQlSqBEmt7hmkYxfT6m,Fortnight (feat. Post Malone),Post Malone,THE TORTURED POETS DEPARTMENT,2024-04-18
2,2qSkIjg1o9h3YT9RAgYN75,Espresso,Sabrina Carpenter,Espresso,2024-04-12
3,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,Artemas,i like the way you kiss me,2024-03-19
4,6XjDF6nds4DE2BBbagZol6,Gata Only,FloyyMenor,Gata Only,2024-02-02


In [4]:
songdetails.head()

,spotify_id,is_explicit,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2OzhQlSqBEmt7hmkYxfT6m,False,228965,0.675,0.397,11,-10.895,1,0.0245,0.499000,0.000006,0.0939,0.319,95.988,4
1,2qSkIjg1o9h3YT9RAgYN75,True,175459,0.701,0.771,0,-5.486,1,0.0293,0.133000,0.000012,0.1970,0.661,103.982,4
2,2GxrNKugF82CnoRFbQfzPf,False,142514,0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4
3,6XjDF6nds4DE2BBbagZol6,True,222000,0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4
4,5Z0UnEtpLDQyYlWwgi8m9C,False,251424,0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4


In [5]:
songdailyrank.head()

,spotify_id,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity
0,2OzhQlSqBEmt7hmkYxfT6m,1,0,49,Global Top 50,2024-04-28,93
1,2qSkIjg1o9h3YT9RAgYN75,2,0,48,Global Top 50,2024-04-28,94
2,2GxrNKugF82CnoRFbQfzPf,3,0,47,Global Top 50,2024-04-28,100
3,6XjDF6nds4DE2BBbagZol6,4,0,46,Global Top 50,2024-04-28,98
4,5Z0UnEtpLDQyYlWwgi8m9C,5,1,45,Global Top 50,2024-04-28,89


In [6]:
# All datas
# Mean and Median calculat
avg_details = songdetails.mean()
median_details = songdetails.median()

# Mean and median in same dataframe
all_details = pd.concat([avg_details, median_details], axis=1)
all_details.columns = ['mean', 'median']

# Calculation of differences
def calculate_percentage_difference(row):
    if row.name in ['duration_ms', 'key', 'loudness', 'tempo', 'time_signature']:
        return ((row['mean'] - row['median']) / row['mean']) * 100
    else:
        return row['mean'] - row['median']

all_details['percentage_difference_(%)'] = all_details.apply(calculate_percentage_difference, axis=1)

column_name = ['mean', 'median', 'percentage_difference_(%)']

# Mean and median values rounded to 2 decimal places
for column in column_name:
    all_details[column] = all_details[column].round(2)

# convert duration_ms to minutes and seconds and update in column
def convert_duration_ms(ms):
    minutes = ms // 60000
    seconds = (ms % 60000) / 1000
    return f"{int(minutes)}:{seconds:.2f}"

column_list = ['mean', 'median'] # List of columns

for column in column_list:
    all_details.loc['duration_ms', column] = convert_duration_ms(all_details.at['duration_ms', column])

# Show
all_details

,mean,median,percentage_difference_(%)
is_explicit,0.33,0,0.33
duration_ms,3:11.72,3:3.00,4.55
danceability,0.67,0.69,-0.02
energy,0.64,0.66,-0.02
key,5.36,6,-11.84
loudness,-7.08,-6.66,5.90
mode,0.51,1,-0.49
speechiness,0.12,0.07,0.05
acousticness,0.28,0.2,0.08
instrumentalness,0.02,0,0.02


In [7]:
# Song details by country
df_rank_details = pd.merge(songdailyrank[['spotify_id', 'country']], songdetails, how='left', on='spotify_id')
df_rank_details = df_rank_details.drop_duplicates(subset=['spotify_id', 'country'])

# Mean by country
mean_df_rank_details = df_rank_details.groupby(['country']).mean().round(3)

# Median by country
median_df_rank_details = df_rank_details.groupby(['country']).median().round(3)

In [8]:
df_rank_details.head()

,spotify_id,country,is_explicit,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2OzhQlSqBEmt7hmkYxfT6m,Global Top 50,False,228965,0.675,0.397,11,-10.895,1,0.0245,0.499000,0.000006,0.0939,0.319,95.988,4
1,2qSkIjg1o9h3YT9RAgYN75,Global Top 50,True,175459,0.701,0.771,0,-5.486,1,0.0293,0.133000,0.000012,0.1970,0.661,103.982,4
2,2GxrNKugF82CnoRFbQfzPf,Global Top 50,False,142514,0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,4
3,6XjDF6nds4DE2BBbagZol6,Global Top 50,True,222000,0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,4
4,5Z0UnEtpLDQyYlWwgi8m9C,Global Top 50,False,251424,0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,4


In [9]:
mean_df_rank_details.head()

,is_explicit,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
country,,,,,,,,,,,,,,
AE,0.384,203482.607,0.641,0.620,4.911,-7.023,0.621,0.093,0.271,0.018,0.187,0.478,119.710,3.905
AR,0.308,194230.179,0.692,0.659,5.856,-6.036,0.533,0.094,0.266,0.014,0.193,0.582,115.471,3.944
AT,0.341,185282.110,0.668,0.638,5.138,-7.263,0.588,0.120,0.247,0.016,0.183,0.498,125.510,3.950
AU,0.320,207418.528,0.619,0.621,4.985,-7.238,0.736,0.075,0.288,0.009,0.192,0.499,123.356,3.903
BE,0.313,200048.475,0.632,0.631,4.814,-7.192,0.620,0.089,0.302,0.016,0.180,0.511,123.802,3.916


In [10]:
median_df_rank_details.head()

,is_explicit,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
country,,,,,,,,,,,,,,
AE,False,206019.0,0.647,0.635,5.0,-6.568,1.0,0.055,0.164,0.0,0.130,0.456,118.026,4.0
AR,False,184800.0,0.713,0.677,6.0,-5.417,1.0,0.063,0.181,0.0,0.139,0.571,102.984,4.0
AT,False,176638.5,0.676,0.650,5.0,-6.940,1.0,0.065,0.152,0.0,0.123,0.484,125.024,4.0
AU,False,209577.0,0.628,0.643,5.0,-6.751,1.0,0.050,0.157,0.0,0.125,0.470,120.811,4.0
BE,False,199433.0,0.644,0.664,5.0,-6.474,1.0,0.053,0.202,0.0,0.118,0.510,122.758,4.0


In [11]:
# Write to CSV file
# Mean
mean_df_rank_details.to_csv('/home/hunor/hobby_projects/spotify/mean_df_rank_details.csv', sep=',', index=True)

# Median
median_df_rank_details.to_csv('/home/hunor/hobby_projects/spotify/median_df_rank_details.csv', sep=',', index=True)

# All details
df_rank_details.to_csv('/home/hunor/hobby_projects/spotify/df_rank_details.csv', sep=',', index=False)